# Install packages

In [ ]:
%%time
#####################################################################################################################
# Copy uploaded packages to working directory
! cp -r ../input/sparknlp-3-1-0-installation/sparknlp_installation.xyz .
! mv sparknlp_installation.xyz sparknlp_installation.tar.gz

# Unzip the package & and remove *.tar
! tar -xzvf sparknlp_installation.tar.gz > /dev/null
! rm -rf ./sparknlp_installation.tar.gz
# Copy spark 3.0.2
! cp -r /kaggle/working/sparknlp_installation/spark-3.0.2-bin-hadoop2.7 /usr/local/
! rm -rf /kaggle/working/sparknlp_installation/spark-3.0.2-bin-hadoop2.7
######################################################################################################################
# Copy Java to prepare Java_home, and sparknlp *.jar dependencies
! cp -r sparknlp_installation/java-8-openjdk-amd64 /usr/lib/jvm
! rm -rf sparknlp_installation/java-8-openjdk-amd64
# If CPU used
#! cp -r ./offline_kaggle_installation/sparknlp_cpu_jar_packages/*.jar ./offline_kaggle_installation/spark-3.0.2-bin-hadoop2.7/jars
# If GPU used
! cp -r ./sparknlp_installation/sparknlp_gpu_jar_packages/*.jar /usr/local/spark-3.0.2-bin-hadoop2.7/jars
! rm -rf ./sparknlp_installation/sparknlp_cpu_jar_packages
! rm -rf ./sparknlp_installation/sparknlp_gpu_jar_packages
! rm -rf ./sparknlp_installation/cache_pretrained/bert_base_cased_en_2.6.0_2.4_1598340336670
! rm -rf ./sparknlp_installation/cache_pretrained/ner_dl_bert_en_2.6.0_2.4_1599550979101
! rm -rf ./sparknlp_installation/cache_pretrained/ner_dl_en_2.4.3_2.4_1584624950746
! rm -rf ./sparknlp_installation/nltk_data/corpora
! rm -rf ./sparknlp_installation/nltk_data/taggers
! rm -rf ./sparknlp_installation/nltk_data/tokenizers/punkt.zip
! mv ./sparknlp_installation/nltk_data/tokenizers/punkt/english.pickle ./sparknlp_installation/nltk_data/tokenizers/punkt/english
! mv ./sparknlp_installation/nltk_data/tokenizers/punkt/PY3/english.pickle ./sparknlp_installation/nltk_data/tokenizers/punkt/PY3/english
! rm -rf ./sparknlp_installation/nltk_data/tokenizers/punkt/*.pickle
! rm -rf ./sparknlp_installation/nltk_data/tokenizers/punkt/PY3/*.pickle
#! rm -rf ./offline_kaggle_installation/nltk_data/tokenizers/punkt/PY3
! rm -rf ./sparknlp_installation/nltk_data/tokenizers/punkt/README
! mv ./sparknlp_installation/nltk_data/tokenizers/punkt/english ./sparknlp_installation/nltk_data/tokenizers/punkt/english.pickle
! mv ./sparknlp_installation/nltk_data/tokenizers/punkt/PY3/english ./sparknlp_installation/nltk_data/tokenizers/punkt/PY3/english.pickle
#! cp ../input/modin-installation/modin-0.9.1-py3-none-manylinux1_x86_64.whl ./offline_kaggle_installation/sparknlp_pyspark_installation
#####################################################################################################################
# Install py4j, pyspark, sparknlp, findspark
!pip install py4j --no-index --find-links=file:///kaggle/working/sparknlp_installation/sparknlp_pyspark_installation
!pip install pyspark --no-index --find-links=file:///kaggle/working/sparknlp_installation/sparknlp_pyspark_installation
!pip install spark_nlp --no-index --find-links=file:///kaggle/working/sparknlp_installation/sparknlp_pyspark_installation
!pip install findspark --no-index --find-links=file:///kaggle/working/sparknlp_installation/sparknlp_pyspark_installation
#!pip install modin --no-index --find-links=file:///kaggle/working/offline_kaggle_installation/sparknlp_pyspark_installation
! rm -r ./sparknlp_installation/sparknlp_pyspark_installation
# ## For model train version 2
# ! cp -r ../input/light-glove-dataset-model/light_dataset_glove_model .
## For model train version 3
#! cp -r ../input/best-dataset-glove-model/dataset_glove_model .

# ## Best version up to now
# ! cp -r ../input/v2-better-glove-dataset-model/dataset_glove_model .
# A light version
#! cp -r ../input/v1-small-dataset-glove-model/small_dataset_glove_model .

! cp -r ../input/kagge-glove-model-v12/kagge_glove_model_v12 .

# Start SparkSession

In [ ]:
%%time
# Setup environment and check java version
import os
%env JAVA_HOME=/usr/lib/jvm/java-8-openjdk-amd64
%env SPARK_HOME=/usr/local/spark-3.0.2-bin-hadoop2.7
os.environ["PYSPARK_SUBMIT_ARGS"]="--master local[*] pyspark-shell"
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/usr/local/spark-3.0.2-bin-hadoop2.7"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
os.environ["PATH"] = os.environ["SPARK_HOME"] + "/bin:" + os.environ["PATH"]
os.environ["LD_LIBRARY_PATH"]="/usr/local/cuda-11.0/lib64:"+os.environ["LD_LIBRARY_PATH"]
os.environ["PATH"]="/usr/local/cuda-11.0/bin:"+os.environ["PATH"]
import findspark
findspark.init()
import sparknlp
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, regexp_replace, input_file_name, rand
from pyspark.sql import functions as F
from pyspark.sql.types import *
from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *
from sparknlp.pretrained import PretrainedPipeline, PipelineModel
from pyspark.ml import Pipeline
from sparknlp.base import LightPipeline

spark = SparkSession.builder \
    .appName("Spark NLP")\
    .master("local[*]")\
    .config("spark.driver.memory","16G")\
    .config("spark.driver.maxResultSize", "0") \
    .config("spark.kryoserializer.buffer.max", "2000M")\
    .config("spark.sql.execution.arrow.pyspark.enabled", "true")\
    .config("spark.sql.execution.arrow.pyspark.fallback.enabled", "true")\
    .config("spark.cleaner.referenceTracking.cleanCheckpoints", True)\
    .config("spark.streaming.stopGracefullyOnShutdown", True)\
    .getOrCreate()

! java -version

print("Spark NLP version: {}".format(sparknlp.version()))
print("Apache Spark version: {}".format(spark.version))

In [ ]:
import os, json, re, gc
import pandas as pd
import nltk
# nltk.load('./offline_kaggle_installation/nltk_data/tokenizers/punkt/english.pickle')
# from nltk.tokenize import sent_tokenize
import pyarrow.parquet as pq
import pyarrow as pa
import pandas as pd
import numpy as np
from tqdm.autonotebook import tqdm
tqdm.pandas()
import dask.dataframe as dd
import multiprocessing as mp

# Set pipeline

In [ ]:
document = DocumentAssembler()\
        .setInputCol("text")\
        .setOutputCol("document")

# sentence = SentenceDetector()\
#         .setInputCols(['document'])\
#         .setOutputCol('sentence')

token = Tokenizer()\
        .setInputCols(['document'])\
        .setOutputCol('token')

glove_embeddings = WordEmbeddingsModel\
        .load('./sparknlp_installation/cache_pretrained/glove_100d_en_2.4.0_2.4_1579690104032') \
        .setInputCols(["document", "token"])\
        .setOutputCol("embeddings")
  
loaded_ner_model = NerDLModel.load("./kagge_glove_model_v12")\
        .setInputCols(["document", "token", "embeddings"])\
        .setOutputCol("ner")

converter = NerConverter()\
        .setInputCols(["document", "token", "ner"])\
        .setOutputCol("ner_chunk")


ner_prediction_pipeline = Pipeline(stages = [
        document,
        token,
        glove_embeddings,
        loaded_ner_model,
        converter
])


empty_df = spark.createDataFrame([['']]).toDF('text')

pipeline_model = ner_prediction_pipeline.fit(empty_df)
lightPipeline = LightPipeline(pipeline_model)


# UDF functions

In [ ]:
# def sentence_detector(text):
#     return sent_tokenize(text)

pd.set_option('display.max_columns', 150)

def sentence_detector(t):
    t = str(t)
    if(len(t) < 5):
        return ['']
    else:
        # split .
        a = re.split('\.', t)   # split .
        a = [x+'.' for x in a if x]   # add . in the last character of sentence
        # find the sentence that length of them < 13
        ind = [0]
        for x in range(1, len(a)):
            if len(a[x]) < 13:
                ind.append(1)
            else:
                ind.append(0)
        # take the index of them
        indices = [i for i, x in enumerate(ind) if x == 0]
        # distributed the sentence of paragraph on indices
        take = []
        if len(indices) == 1:
            take.append([0, len(ind)])
        else:
            for i in range(len(indices)):
                if i == len(indices)-1:
                    take.append([indices[i], len(ind)])
                else:
                    take.append([indices[i], indices[i+1]])
        # take the array of sentence before split . 
        b = []
        for x in take:
            tem = ''.join(a[x[0]:x[1]])
            b.append(tem)
        a = b
        # find the sentence that not start by space or new line and followed is upper character
        ind = [0]
        for x in range(1, len(a)):
            if re.match(' [A-Z]|\n[A-Z]', a[x][:2]):
                ind.append(0)
            else:
                ind.append(1)
        # take the index of them    
        indices = [i for i, x in enumerate(ind) if x == 0]
        # distributed the sentence of paragraph on indices
        take = []
        if len(indices) == 1:
            take.append([0, len(ind)])
        else:
            for i in range(len(indices)):
                if i == len(indices)-1:
                    take.append([indices[i], len(ind)])
                else:
                    take.append([indices[i], indices[i+1]])
        # take the array of sentence before split .
        b = []
        for x in take:
            tem = ''.join(a[x[0]:x[1]])
            b.append(tem)
        return b

udf_sentence_detector = udf(sentence_detector, ArrayType(StringType()))

def get_ner(text):
    entity = []
    for ann in lightPipeline.fullAnnotate(text)[0]['ner_chunk']:
        entity.append([ann.result, ann.metadata['entity'], ann.metadata['confidence']])
    return entity

def containsAny(string, array):
    if len(string) == 0:
        return False
    else:
        return (any(word in string for word in array))

contains_udf = udf(containsAny, BooleanType())

def clean_text(txt):
    return re.sub('[^A-Za-z0-9]+', ' ', str(txt).lower()).strip()

udf_clean_text = udf(clean_text, StringType())

def jaccard(str1, str2):
    a = set(str1.lower().split())
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

def filterSentence(sent):
    sent = re.sub('XXXY|YXXX|XXXX|XXYY|YYYY', '', sent)
    regex = r"[A-Z][A-Za-z]+"
    matches = re.finditer(regex, sent)
    words = {}
    for matchNum, match in enumerate(matches):
        words[f'{match.group()}'] = [match.start(), match.end()]
#     print(words)
    k = list(words.keys())
    v = list(words.values())     
    for x in k:
        if re.match('[A-Z]{3,}', x):
            return True
    count = 0
    for x in range(1, len(v)):
        if abs(v[x][0] - v[x-1][1]) < 8:
            count = count + 1
            if count >= 1:
                return True
    return False

udf_filter_sentence = udf(filterSentence, BooleanType())

# Load json text

In [ ]:
%%time
dir_train_path = '../input/coleridgeinitiative-show-us-the-data/train/'
dir_test_path = '../input/coleridgeinitiative-show-us-the-data/test/'


def get_paragraph(dirpath, arr_data, fileid):
    with open('{}'.format(dirpath+fileid)) as f:
        t = json.load(f)
#     i = max_id_to_index + 1
    for x in t:
        fileid = re.sub(".json", "", fileid)
        text = str(x['text'])
        # remove text in section that is very difficult to say using what data
        section_removal = ["unknown"]
        #section_removal = ["result", "results", "appendix", "conclusion", "limitations", 
        #                   "limitation", "supplementary", "summary", "reference", "references"]
        section = x['section_title'].lower().split()
        if (not any(s in section_removal for s in section) & (len(text) > 35)):
            emoji_pattern = re.compile("["
                                   u"\U0001F600-\U0001F64F"  # emoticons
                                   u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                                   u"\U0001F680-\U0001F6FF"  # transport & map symbols
                                   u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                                   "]+", flags=re.UNICODE)
            text = emoji_pattern.sub(r'', text)
            arr_data.append([fileid, text])
#             i+=1
            
# Start collecting test json publications and save it to a parquet file
fileids = os.listdir(dir_train_path)[1:500]
#fileids = ["bde73ad6-ce46-489b-8ee0-b162698cc6df.json"]
arr_data = []
i = 0
for fileid in fileids:
    get_paragraph(dir_train_path, arr_data, fileid)
    #i = len(arr_data)
    
table = pa.Table.from_pandas(pd.DataFrame(arr_data, columns=['Id','text']), preserve_index=False)
pq.write_table(table, 'publication_text_test.parquet')

del [[table, arr_data]]
gc.collect()

# The objective of golden words is to reduce computing dimension and higher the accuracy
#golden_words = pd.read_csv("../input/golden-words/golden_words.csv").golden_words.tolist()
#golden_words = ["data", "codes", "database", "sample", "dataset", "survey", "using"]
golden_words = ["data", "codes", "database", "sample", "sampling", "datasets", "survey", "dataset", "iri", \
                "overland","samples", "registry", "baseline", "temperature", "genome", "census", "noaa", "world", \
                "baltimore", "hub", "initiative", "international", "national", "baccalaureate", "beginning", \
                "coastal", "practices", "cccsl", "tide", "adni", "slosh", "c-cap", "ccap", "research", "covid", \
                "education", "aging", "longtitudinal", "science", "disease", "school", "doctorates"]

# golden_words = ["data", "codes", "database", "sample", "samples", "sampling", "datasets", "survey", "dataset", "iri", "hurricanes",\
#                 "samples", "registry", "baseline", "temperature", "genome", "census", "noaa", "world", "baltimore", "hub",\
#                 "initiative", "alzheimer", "international", "national", "baccalaureate", "beginning", "coastal", "practices", "cccsl",\
#                 "program", "study", "network", "catalog", "tide", "adni", "from"]
# golden_verbs = ["were used", "using", "was used", "used","derived from", "derived", "collected from", "collected", \
#                "conducted", "gathered", "gathered from", "surveyed", "retrieved", "retrieved from"\
#               "extracted from", "obtained", "obtained from", "downloaded", "downloaded from"]
golden_verbs = ["used", "using", "uses", "derives", "derived", "collecte", "collected", "collecting",\
                "conducts", "conducted", "conducts", "conducting", "gather", "gathered", "gathering", \
                "surveyed", "retrieved", "retrieve", "retrieving", "retrieves","received", "receives", "receiving",
               "extracted", "extracting", "extract", "obtained", "obtaining", "downloaded", "downloading", \
                "acquired", "acquires", "acquiring", "acquire", "based"]
                
golden_words = golden_words + golden_verbs
#print(golden_words)
data_to_test = spark.read.parquet("publication_text_test.parquet")\
.filter(col("text").isNotNull())\
.withColumn("length_sent", F.length(col("text"))).filter(col("length_sent") > 35)\
.withColumn("sent", udf_sentence_detector("text"))\
.select("Id", F.explode(col("sent")).alias("text"))\
.withColumn("text1", F.regexp_replace(col("text"), "[^0-9a-zA-Z]+", " "))\
.withColumn("token", F.split(F.lower(col("text1")), ' '))\
.withColumn("has_golden_words", contains_udf(col("token"), F.array([F.lit(i) for i in golden_words])))\
.filter(col("has_golden_words") == "true")\
.withColumn("issentence", udf_filter_sentence(col("text")))\
.filter(col("issentence") == True)\
.select("Id", "text")\
.drop_duplicates()

data_to_test.repartition(380).write.mode("overwrite").format("parquet").save("cleaned_test_data")

# Truncate data to 2 parts

In [ ]:
%%time
data_to_test_part1 = spark.read.parquet("cleaned_test_data")\
.filter(F.size(F.split(col("text"), " ")) < 250)

data_to_test_part1.repartition(380).write.mode("overwrite").format("parquet").save("cleaned_test_data1")

In [ ]:
%%time
data_to_test_part2 = spark.read.parquet("cleaned_test_data")\
.filter(F.size(F.split(col("text"), " ")) >250)\
.withColumn("text", F.regexp_replace(col("text"), "\s", " "))\
.withColumn("text", F.regexp_replace(col("text"), "[A-Za-z]\d+|\d+[A-Za-z]|\d+|[/\\&\.()<>\-,%\+\*=\]\]\'\"]", ""))\
.withColumn("text", F.regexp_replace(col("text"), r"\b\w{1,2}\b", " "))\
.withColumn("text", F.regexp_replace(col("text"), " {2,}", " "))

data_to_test_part2.repartition(380).write.mode("overwrite").format("parquet").save("cleaned_test_data2")

# Get dataset name

In [ ]:
%%time
df1 = spark.read.parquet("cleaned_test_data1")
df2 = spark.read.parquet("cleaned_test_data2")

pipeline_model.transform(df1).write.mode("overwrite").parquet("ner_model_output1")
pipeline_model.transform(df2).write.mode("overwrite").parquet("ner_model_output2")
#pipeline_model.transform(df).show()

df1_output = spark.read.parquet("ner_model_output1")
df2_output = spark.read.parquet("ner_model_output2")
df_output = df1_output.union(df2_output)
df_output.write.mode("overwrite").parquet("ner_model_output")


# Preprocess from first NER output

In [ ]:
%%time
df1 = spark.read.parquet("ner_model_output")
#df1.printSchema()
df1.withColumn("tmp", F.explode(col("ner_chunk")))\
.select("Id", "text","tmp.*").select("Id", "text", "result", "metadata.entity", "metadata.confidence")\
.withColumnRenamed("result", "ner_extracted")\
.withColumnRenamed("entity", "ner_label").write.mode("overwrite").parquet("ner_output")

In [ ]:
%%time
df_input_rest = pd.read_parquet("ner_output", engine='pyarrow')
# df_input_rest_ = pd.read_parquet("cleaned_test_data", engine='pyarrow')
# df_input_rest = dd.from_pandas(df_input_rest_, npartitions=16*mp.cpu_count())
# df_input_rest["entity"] = df_input_rest.map_partitions(lambda df: df.apply(lambda x: get_ner(x.text), axis=1), meta =("entity",str))
# df_input_rest = df_input_rest.compute(num_workers=4)
# df_input_rest = df_input_rest[["Id", "text", "entity"]]
# df_input_rest = df_input_rest.explode("entity")
# df_input_rest = df_input_rest[df_input_rest.entity.notnull()]

# df_input_rest['ner_extracted'] = df_input_rest.entity.str[0]
# df_input_rest['ner_label'] = df_input_rest.entity.str[1]
# df_input_rest['confidence'] = df_input_rest.entity.str[2]
#df_input_rest =df_input_rest[df_input_rest.ner_label == "DATASET_TITLE"]
df_input_rest = df_input_rest[["Id", "text", "ner_extracted", "ner_label", "confidence"]]
#df_input_rest =df_input_rest[df_input_rest.ner_label == "DATASET_TITLE"]
df_input_rest = df_input_rest.rename(columns={"entity":"ner_extracted"})
raw_output_ = df_input_rest.drop_duplicates(keep="first")
del [[df_input_rest]]
gc.collect()

# Output Correction

In [ ]:
train = pd.read_csv('../input/coleridgeinitiative-show-us-the-data/train.csv')
dataset_title = list(train['dataset_title'])
dataset_label = list(train['dataset_label'])
cleaned_label = list(train['cleaned_label'])
#all_labels = list(set(list(map(clean_text, set(dataset_title))) + list(map(clean_text, set(dataset_label)))))
all_labels = list(set(dataset_title+dataset_label+cleaned_label))
all_labels.append("Alzheimer's Disease Neuroimaging Initiative ADNI")
all_labels.append("SLOSH")
all_labels.append("NOAA C-CAP")
all_labels.append("NOAA CCAP")
all_labels.append("NOAA OISST")
all_labels.append("OASIS")
all_labels.append("HERD")
all_labels.append("National Inpatient Sample")
all_labels.append("Alzheimer's Disease Neuroimaging Initiative")
all_labels.append("Program for International Student Assessment")
all_labels.append("Progress in International Reading Literacy Study")
all_labels.append("SARS‐CoV‐2 genome sequences")
print("Length of all_labels: ", len(all_labels))
# Check if words has at least 2 capital letters
def isAllCap(words):
    if sum([int(w.isupper()) for w in words]) > 1:
        return True
    else:
        return False
#print("test isAllCap:", isAllCap("LINH"))
# jaccard
def jaccard(str1, str2):
    a = set(str1.lower().split())
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

# calculate max jaccard of string with train all dataset lables
def max_jaccard_train(text):
    #max_score = 0
    if (text == None):
        return 0
    else:
        jac_score = []
        for label in all_labels:
            jac_score.append(float(jaccard(text, label)))
    return float(max(jac_score))

#print(max_jaccard_train("covid"))
# Filter non-potential dataset
# For example: Analytic data -> so it cannot be a dataset
def drop_non_potential_dataset(text):
    nbr_words = text.split()
    if ((text==")") | (text=="(")):
        return None
    elif ((len(nbr_words) == 2) & ("data" in nbr_words)):
        nbr_words.remove("data")
        if isAllCap(nbr_words[0]):
            return text
        else: return None
    else: return text
        
#print(drop_non_potential_dataset("BHB data"))

# correction, for example: and Overland Surges from Hurricanes

# correction, for example: and Overland Surges from Hurricanes
def label_correction(txt):
    if txt == None:
        return None
    else:
        text = txt.lower()
    if "overland surges from hurricanes" in text:
        return "NOAA Sea, Lake, and Overland Surges from Hurricanes"
    if "and overland surges" in text:
        return "NOAA Sea, Lake, and Overland Surges from Hurricanes"
    elif "beginning postsecondary" in text:
        return "Beginning Postsecondary Student"
    elif "genome sequence of s" in text:
        return "SARS-CoV-2 genome Sequence"
    elif "genome sequences of s" in text:
        return "SARS-CoV-2 genome Sequences"
    elif text.lower() == "noaa tide":
        return "NOAA tidal Station"
    elif "continuum codes" in text:
        return "Rural-Urban Continuum Codes"
    elif "neuroimaging initiative" in text:
        return "Alzheimer's Disease Neuroimaging Initiative ADNI"
    elif "adni" in text:
        return "ADNI"
    elif (("adni" in text) & ("initiative" not in text)):
        return "ADNI"
    elif ((len(text.split()) < 4) & ("alzheimer" in text)):
        return "ADNI"
    elif "storm surge inundation" in text:
        return "NOAA Storm Surge Inundation"
    elif "sea surface temperature" in text:
        return "Optimum Interpolation Interpolation Sea Surface Temperature"
    elif "oisst" in text:
        return "Optimum Interpolation Interpolation Sea Surface Temperature"
    elif "postsecondary student" in text:
        return "Beginning Postsecondary Students"
    elif "timss" in text:
        return "Trends in International Mathematics and Science Study"
    elif "pirls" in text:
        return "PIRLS"
    elif "ccap" in text:
        return "NOAA C-CAP"
    elif "c-cap" in text:
        return 'NOAA C-CAP'
    elif "iri" in text.split():
        return "IRI Consumer Network Panel"
    elif "cnp" in text.split():
        return "IRI Consumer Network Panel"
    elif "pisa" in text.split():
        return "Program for International Student Assessment"
    elif "slosh" in text:
        return "SLOSH Model"
    elif "herd" in text:
        return "Higher Education Research and Development Survey"
    elif "national assessment of education" in text:
        return "National Assessment of Education Progress"
    elif "trends in international mathematics" in text:
        return "Trends in International Mathematics and Science Study"
    elif text == "storm surge modeling":
        return "slosh Model"
    elif "nels" in text:
        return "National Education Longitudinal Study"
    elif text == "blsa":
        return "Baltimore Longitudinal Study of Aging"
    elif text == "nels":
        return "National Education Longitudinal Study"
    elif text == "naep":
        return "National Assessment of Education Progress"
    elif "staffing survey" in text:
        return "Schools and Staffing Survey"
    elif text == "schools and":
        return "Schools and Staffing Survey"
    elif "survey of earned doctorates" in text:
        return "Survey of Earned Doctorates"
    elif "whole-genome sequence of" in text:
        return "Genome Sequence of SARS-COV-2"
    elif "appendix" in text:
        return None
    elif "use" in text:
        return None
    elif "uses" in text:
        return None
    elif "using" in text:
        return None
        return None
    elif "]" in text:
        return None
    elif "\n" in text:
        return None
    elif "esi shoreline" in text:
        return "ESI Shoreline"
    elif text == "tide":
        return None
    elif "tide gauge" in text:
        return "NOAA Tide Gauge"
    elif text == "noaa":
        return None
    else:
        return txt
    
    
def voting(text):
    text_list = text.split("|")
    voting = text_list
    if (len(text_list)) > 1:
        for dt in text_list:
            if "data" in dt.split():
                new_df = dt.replace("data", "database")
                for dt1 in text_list:
                    if new_df == dt1:
                        voting.remove(dt)
                new_df = dt.replace("data", "dataset")
                for dt1 in text_list:
                    if new_df == dt1:
                        voting.remove(dt)
            if "database" in dt.split():
                new_df = dt.replace(" database", "")
                for dt1 in text_list:
                    if new_df == dt1:
                        voting.remove(dt)
                new_df = dt.replace("database", "dataset")
                for dt1 in text_list:
                    if new_df == dt1:
                        voting.remove(dt)
        text_list = voting
        for dt in text_list:
            if "data" in dt.split():
                new_df = dt.replace(" data", "")
                for dt1 in text_list:
                    if new_df == dt1:
                        voting.remove(dt)
    return "|".join([v for v in voting if v])

#voting("early childhood longitudinal study|ecls k database|ecls k data")

def isAllLower(text):
    word_list = text.split()
    if sum([int(w[0].isupper()) for w in word_list]) == 0:
        return True
    elif "moisture" in word_list:
        return False
    elif "maps" in word_list:
        return False
    elif ((word_list[-1] == "survey") & (len(word_list) > 2)):
        return False
    elif word_list[-1][0].isupper() == False:
        return True
    elif ((len(word_list) == 1) & (len(word_list[0]) > 6)):
        return True
    elif ((len(word_list) == 3) & (sum([int(w[0].isupper()) for w in word_list[0:1]]) == 0)):
        return True
    else:
        return False
    
#isAllLower("For School achievement")

In [ ]:
# pd.set_option('display.max_columns', 150, 'display.max_colwidth', None)
# raw_output_ = df_input_rest

In [ ]:
#text = "The data used to support the findings of this study can be obtained from the Alzheimer's Disease Neuroimaging Initiative (ADNI) database (http://adni.loni.usc.edu/)."
#text = "The data for this report were drawn from the first followup of the 1990 Beginning Postsecondary Students (BPS) Longitudinal Study conducted in the spring of 1992."
#text = "In addition to county centroid elevation and total county population as an offset term, each model contained the following covariates: 1) 9-level USDA 2013 Rural-Urban Continuum Codes (USDA RUCC), where higher-numbered categories indicate increasingly rural environments; 2) average number of persons per household, computed as county population/number of households; 3) interaction of persons per household and USDA Rural-Urban Continuum Codes; 4) independent state random effects; and 5) spatially-correlated county random effects."
#text = "In such cases, attribution based on HURDAT2 (or IBTrACS) alone will possibly introduce some uncertainties along the western coast section where TCs from both sides may coincide, and the effect is likely a weakening of the fluxes by coinciding TCs from both sides."
#text = "The Beginning Postsecondary Student also discusses mitigation activities that are currently being used throughout the State and potential strategies that should be considered in the future."
#text = "In this analysis, we used the Progress in International reading Literacy Study (PIrLS) to analyze images"

#text = "Storm surge modeling data was obtained from the SLOSH display program and used to map the extent and depth of inundation resulting from various storm scenarios ( Figure 6)."
#text = "Coastal flooding is the hazard with the longest history of hazard mitigation planning throughout the study region, primarily through participation in the National Flood Insurance Program (NFIP), which is administered by the Federal Emergency Management Agency (FEMA)."
# text ="Comparative Indicators of Education in the United States and Other G-8 Countries: 2009 draws on the most current information about education from four primary sources: the Indicators of National Education Systems (INES) at the Organization for Economic Cooperation and Development (OECD); the Program for International Student Assessment (PISA)"
# get_ner(text)

In [ ]:
#raw_output_ = r

# Correction Process

In [ ]:
%%time
pd.set_option('display.max_columns', 150, 'display.max_colwidth', None)
raw_output_["confidence"] = pd.to_numeric(raw_output_["confidence"], downcast="float")
raw_output_["ner"] = raw_output_.ner_extracted.apply(drop_non_potential_dataset)
raw_output_["ner"] = raw_output_.ner.apply(label_correction)
raw_output_ = raw_output_[raw_output_.ner.notnull()]
raw_output_.drop("ner_extracted", axis="columns", inplace=True)
#raw_output["max_jaccard"] = raw_output.ner.apply(max_jaccard_train)
raw_output = dd.from_pandas(raw_output_, npartitions=4*mp.cpu_count()) 
raw_output["max_jaccard"] = raw_output.map_partitions(lambda df: df.apply(lambda x: max_jaccard_train(x.ner), axis=1), meta=("max_jaccard", float))
#df_input_rest = dd.from_pandas(df_input_rest_, npartitions=1*mp.cpu_count())
#df_input_rest["entity"] = df_input_rest.map_partitions(lambda df: df.progress_apply(lambda x: get_ner(x.text), axis=1), meta =("entity",str))
raw_output = raw_output.compute(num_workers=4)
# del [[raw_output_]]
# gc.collect()

In [ ]:
raw_output["isAllLower"] = raw_output.ner.apply(isAllLower)
#raw_output = raw_output[(raw_output.isAllLower == False) & (raw_output.max_jaccard < 0.5)]
#raw_output.drop('isAllLower', axis=1, inplace=True)
#raw_output[raw_output.Id == "b5282251-0d12-4a33-b332-84ae7128d3e0"]

In [ ]:
# %%time
# raw_output = df_input_rest
# raw_output["confidence"] = pd.to_numeric(raw_output["confidence"], downcast="float")
# raw_output["ner"] = raw_output.ner_extracted.apply(drop_non_potential_dataset)
# raw_output["ner"] = raw_output.ner.apply(label_correction)
# raw_output = raw_output[raw_output.ner.notnull()]
# raw_output.drop("ner_extracted", axis="columns", inplace=True)
# raw_output["max_jaccard"] = raw_output.progress_apply(lambda x: max_jaccard_train(x.ner), axis=1)

In [ ]:
def filter_ner(text, confidence):
    words = text.lower().split()
    tf_return = False
    if ((len(words) > 2) & ("survey" in text.lower())):
        tf_return = True
    if ((len(words) > 2) & ("dataset" in text.lower())):
        tf_return = True
    if ((len(words) > 2) & ("datasets" in text.lower())):
        tf_return = True
    if ((len(words) > 2) & ("database" in text.lower())):
        tf_return = True
    if ((len(words) > 2) & ("archive" in text.lower())):
        tf_return = True
    if ((len(words) > 2) & ("sample" in text.lower())):
        tf_return = True
    if ((confidence > 0.82)) & (len(words) > 3):
        tf_return = True
    if (len(words) > 8):
        tf_return = False
    return tf_return
        
        
        
#raw_output1 = raw_output[raw_output.ner_label.isin(["DATASET_TITLE", "DATASET_LABEL"])]
# raw_output1 = raw_output1[raw_output1.confidence > 0.6]
raw_output1 = raw_output
raw_output1 = raw_output1[raw_output1.max_jaccard < 0.5]
#raw_output1 = raw_output1[raw_output1.confidence > 0.8]
raw_output1 = raw_output1[raw_output1.isAllLower == False]
raw_output1["filter_1"] = raw_output1.apply(lambda x: filter_ner(x.ner, x.confidence), axis=1)
raw_output1 = raw_output1[raw_output1.filter_1 == True]
raw_output1 = raw_output1[list(raw_output.columns)]
#raw_output.drop('isAllLower', axis=1, inplace=True)


raw_output2 = raw_output[raw_output.max_jaccard >= 0.5]

output = raw_output1.append(raw_output2)
output = output[["Id", "ner"]].drop_duplicates(keep="first")

In [ ]:

def elect_outcome(text):
    words_in_text = text.split("|")
    outcome= []
    if len(words_in_text) < 2:
        return text
    else:
        for x in range(len(words_in_text)):
            for y in range(x+1, len(words_in_text)):
                if ((words_in_text[x] != words_in_text[y]) & (jaccard(words_in_text[x], words_in_text[y]) >= 0.5) & (jaccard(words_in_text[x], words_in_text[y]) != 1)):
                    if len(words_in_text[x]) > len(words_in_text[y]):
                        outcome.append(words_in_text[y])
                        outcome = list(set(outcome))
                    else:
                        outcome.append(words_in_text[x])
                        outcome = list(set(outcome))
        outcome = [w for w in words_in_text if w not in outcome]
        return "|".join([w for w in outcome if w])

             

text = 'adni|alzheimer s disease neuroimaging initiative adni|adrc dataset|wisconsin adrc dataset|w adrc dataset'
                        
print(elect_outcome(text))

In [ ]:
output = output.rename(columns={"ner":"PredictionString"})
output["PredictionString"] = output.PredictionString.apply(clean_text)
output = output[output.PredictionString.notnull()]
output = output.drop_duplicates(keep="first")
output = output.groupby('Id').agg(lambda c: '|'.join(c)).reset_index()
output["PredictionString"] = output.PredictionString.apply(voting)
output["PredictionString"] = output.PredictionString.apply(elect_outcome)

In [ ]:
#raw_output

# Result

In [ ]:
dir_test_path = '../input/coleridgeinitiative-show-us-the-data/test'
files = os.listdir(dir_test_path)
list_id = [re.sub(".json", "", f) for f in files]
submit_std = pd.DataFrame(list_id, columns=["Id_std"])
result = pd.merge(submit_std, output, left_on="Id_std", right_on="Id", how="left")
result = result[["Id_std", "PredictionString"]].rename(columns={"Id_std":"Id"})
result.to_csv('submission.csv', index=False)

pd.set_option('display.max_colwidth', None, 'display.max_rows', None)
output

In [ ]:
#raw_output

In [ ]:
# output_id = list(set(output.Id))
# output_input = list(set(spark.read.parquet("./cleaned_test_data").select("Id").distinct().toPandas()["Id"]))
# rawoutput_id = list(set(raw_output.Id))

# Submission

In [ ]:
# [w for w in output_input if w not in rawoutput_id]

In [ ]:
# [w for w in output_input if w not in output_id]

In [ ]:
# raw_output[raw_output.Id == "e0614647-0606-45bc-9b6b-bcc2fce45b96"]

In [ ]:
# result.to_csv('submission.csv', index=False)
# with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):  # more options can be specified also
#     print(result)
    
# del [[result2, output, result]]
# gc.collect()

In [ ]:
! rm -rf publication_text_test.parquet
! rm -rf ./cleaned_test_data
! rm -rf ./cleaned_test_data1
! rm -rf ./cleaned_test_data2
! rm -rf ./sparknlp_installation
! rm -rf ./ner_model_output1
! rm -rf ./ner_model_output2
! rm -rf ./ner_model_output
! rm -rf ./ner_output
! rm -rf ./kagge_glove_model_v12
! ls .
print("it's done!")